In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Descriptors import ExactMolWt
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit.Chem import Draw
from IPython.display import display, Image
import io
from collections import Counter
#from spellchecker import SpellChecker
from ast import literal_eval
from collections import defaultdict
import sys

In [ ]:
# Some code to remove NaN smiles
def remove_nan_smiles(df):
    print("--- REMOVING NAN SMILES ---")
    orig_len = len(df)
    print(f"\tOriginal size before removing NaN SMILES: {orig_len}")
    df = df.dropna(subset=['IsomericSMILES'])
    print(f"\tNumber of NaN SMILES: {orig_len - len(df)}")
    print(f"\tSize after removing NaN SMILES: {len(df)}")
    return df

# Some code to purge undesirable elements
def contains_undesirable_elements(smiles, undesirable_elements_list):
    undesirable_smarts = [Chem.MolFromSmarts(f'[{element}]') for element in undesirable_elements_list]
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"\tWarning: Could not parse SMILES: {smiles}")
        return True
    for i, smarts in enumerate(undesirable_smarts):
        if mol.HasSubstructMatch(smarts):
            print(f"\tMatch found: SMILES {smiles} contains {undesirable_elements_list[i]}")
            return True
    return False

def remove_undesirable_elements(df):
    print("--- REMOVING UNDESIRABLE ELEMENTS ---")
    print(f"\tOriginal size before removing undesirable elements: {len(df)}")
    undesirable_elements_list = ["Na", "K", "Ca", "Mg", "Cr", "B", "Si", "Ti", "Fe", "Al", "Co", "Zn", "Cu", "V", "Bi", "He"]
    print(f"\tPurging undesirable elements in {undesirable_elements_list}")
    df['contains_undesirable_elements'] = df.apply(lambda row: contains_undesirable_elements(row['IsomericSMILES'], undesirable_elements_list), axis=1)
    print(f"\tTotal undesirable element SMILES removed: {len(df[df['contains_undesirable_elements'] == True])}")
    df_filtered = df[~df['contains_undesirable_elements']].copy()
    df_filtered.drop(columns=['contains_undesirable_elements'], inplace=True)
    return df_filtered


def smiles_to_inchi_key(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return Chem.inchi.MolToInchiKey(mol)
    return None

def remove_duplicate_smiles(df):
    print("--- REMOVING DUPLICATES ---")
    print(f"\tOriginal size before removing duplicates: {len(df)}")
    df['InChIKey'] = df['IsomericSMILES'].apply(smiles_to_inchi_key)
    duplicates = df[df.duplicated(subset='InChIKey', keep='first')]
    for smiles in duplicates['IsomericSMILES']:
        print(f"\tRemoving duplicate smiles: {smiles}")
    df.drop_duplicates(subset=['InChIKey'], keep='first', inplace=True)
    df.drop(columns=['InChIKey'], inplace=True)
    print(f"\tNumber of SMILES removed: {len(duplicates)}")
    print(f"\tSize after removing duplicates: {len(df)}")

    return df


# Some code to lowercase labels and merge them together
def lowercase_and_merge_labels(df, column_list):
    print("--- LOWERCASING AND MERGING LABELS ---")
    print(f"\tLength of original descriptor list before lowercase and merge: {len(column_list)}")
    # Convert all column names to lowercase
    df.rename(columns={col: col.lower() for col in column_list if col in df.columns}, inplace=True)
    
    # Identify lowercase columns that are now duplicates
    lowercase_columns = [col.lower() for col in column_list if col.lower() in df.columns]
    duplicate_columns = set([col for col in lowercase_columns if lowercase_columns.count(col) > 1])
    
    # Merge duplicate columns
    for col in duplicate_columns:
        duplicate_cols = [c for c in df.columns if c == col]
        df[col] = df[duplicate_cols].any(axis=1)
        # Drop all but one of the duplicate columns
        df.drop(columns=duplicate_cols[1:], inplace=True)
    
    # Create new list of columns without duplicates
    new_column_list = [col for col in lowercase_columns if col in df.columns]
    

    # Print non-unique column names
    if duplicate_columns:
        print("\tNon-unique column names after lowercase conversion:")
        for col in duplicate_columns:
            print(col)
    

    print(f"\tLength of descriptor list after lowercase and merge: {len(column_list)}")
    
    return df, new_column_list


# Some code to fix spelling mistakes in labels?
def fix_spelling_mistakes(df, column_list):
    spell = SpellChecker()
    for col in column_list:
        misspelled = spell.unknown(df[col].unique())
        for word in misspelled:
            # Get the one `most likely` answer
            print(f"\tMisspelled word: {word}, possible corrections: {spell.candidates(word)}")
    return df

# Some code to purge low label counts (<20)
def remove_low_count_columns(df, column_list, threshold=40):
    print("--- REMOVING LOW COUNT COLUMNS ---")
    print(f"\tLength of original descriptor list before low count removal: {len(column_list)}")
    columns_to_remove = []
    for column in column_list:
        column_sum = df[column].astype(bool).sum()
        if column_sum < threshold:
            columns_to_remove.append(column)
            print(f"\tRemoving column '{column}' with total count {column_sum}")
    
    df_filtered = df.drop(columns=columns_to_remove)
    if columns_to_remove == []:
        print(f"\tNo columns are below count threshold {threshold}")
    column_list = [col for col in column_list if col not in columns_to_remove]
    print(f"\tLength of original descriptor list after low count removal: {len(column_list)}")
    return df_filtered, column_list

# Some code to remove columns that are fully zero in label_columns
def remove_no_labels(df, column_list):
    print("--- REMOVING SMILES WITH NO LABELS ---")
    print(f"\tLength of original df before no label removal: {len(df)}")
    # Create a mask for rows where all specified columns are zero
    zero_mask = (df[column_list] == 0).all(axis=1)
    
    # Print IsomericSMILES for rows with all zeros in label columns
    for smiles in df.loc[zero_mask, 'IsomericSMILES']:
        print(f"\tRemoving row with IsomericSMILES: {smiles}")
    
    df_filtered = df[~zero_mask].copy()
    # Remove the rows and return the new dataframe
    print(f"\tLength of df after no label removal: {len(df_filtered)}")
    return df_filtered, column_list    

# Some code to purge large MWs or really small MWs

def visualize_molecule(smiles):
    # Convert SMILES to RDKit molecule
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is not None:
        # Generate a 2D depiction of the molecule
        img = Draw.MolToImage(mol)
        
        # Convert the image to bytes
        bio = io.BytesIO()
        img.save(bio, format='PNG')
        
        # Display the image
        display(Image(bio.getvalue()))
    else:
        print(f"\tInvalid SMILES: {smiles}")

def remove_molecular_weights(df, column_list, lower=20, higher=600):
    print(f"--- REMOVING MOLECULAR WEIGHTS BETWEEN {lower} AND {higher} ---")
    print(f"\tOriginal size before molecular weight thresholding: {len(df)}")
    df['rdkit'] = df.apply(lambda row: Chem.MolFromSmiles(row['IsomericSMILES']), axis=1)
    df['mw'] = df.apply(lambda row: ExactMolWt(row['rdkit']), axis=1)

    print(f"\tMolecules below MW threshold {lower}: {len(df[df['mw'] < lower])}")
    for _, row in df[df['mw'] < lower].iterrows():
        positive_columns = [col for col in column_list if row[col] == 1]
        print(f"\t\t{row['IsomericSMILES']}, {', '.join(positive_columns)}")
        #visualize_molecule(row['IsomericSMILES'])

    print(f"\tMolecules above MW threshold {higher}: {len(df[df['mw'] > higher])}")
    for _, row in df[df['mw'] > higher].iterrows():
        positive_columns = [col for col in column_list if row[col] == 1]
        print(f"\t\t{row['IsomericSMILES']}, {', '.join(positive_columns)}")
        #visualize_molecule(row['IsomericSMILES'])
        
    df_filtered = df[(df['mw'] >= lower) & (df['mw'] <= higher)].copy()
    df_filtered.drop(columns=['rdkit', 'mw'], inplace=True)
    print(f"\tSize after molecular weight thresholding: {len(df_filtered)}")
    return df_filtered

# Some code to purge salts or two-molecules-in-one-SMILES

def contains_salts_and_charges(smiles):
    # Try to create an RDKit molecule from the SMILES
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        print(f"\tInvalid SMILES: {smiles}")
        return True

    # Check for charges
    if Chem.GetFormalCharge(mol) != 0:
        print(f"\t{smiles} has charges")
        return True
    return False

def contains_multimolecule(smiles):
    # Try to create an RDKit molecule from the SMILES
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        print(f"\tInvalid SMILES: {smiles}")
        return True
    
    # Check for multiple molecules
    if "." in smiles:
        print(f"\t{smiles} has multiple molecules")
        return True

    return False

def remove_salts_and_charges_and_multimolecule(df):
    print("--- REMOVING SALTS, CHARGED MOLECULES AND MULTIMOLECULES ---")
    print(f"\tOriginal size before removing salts, charges and multimolecules: {len(df)}")
    df['contains_salts_and_charges'] = df['IsomericSMILES'].apply(contains_salts_and_charges)
    print(f"\tTotal salts and charged SMILES removed: {len(df[df['contains_salts_and_charges'] == True])}")
    df_filtered = df[~df['contains_salts_and_charges']].copy()
    df_filtered['contains_multimolecule'] = df['IsomericSMILES'].apply(contains_multimolecule)
    print(f"\tTotal multimolecules removed: {len(df_filtered[df_filtered['contains_multimolecule'] == True])}") 
    df_filtered = df_filtered[~df_filtered['contains_multimolecule']].copy()
    df_filtered.drop(columns=['contains_salts_and_charges', 'contains_multimolecule'], inplace=True)
    print(f"\tFinal size after removing salts, charges and multimolecules: {len(df_filtered)}")
    return df_filtered


In [ ]:
file_df = pd.read_csv("file_cleaning_features.csv")
file_df.loc[file_df['dataset'] == "keller_2016", 'unclean'] = "keller_2016/keller_2016_unclean_processed.csv"
keller_df = pd.read_csv("keller_2016/keller_2016_unclean_processed.csv")
file_df.loc[file_df['dataset'] == "keller_2016", 'label_columns'] = str(keller_df.columns[:-4].to_list())

file_df.loc[file_df['dataset'] == "abraham_2012", 'label_columns'] = str(['Log (1/ODT)'])

file_df.loc[file_df['dataset'] == "mayhew_2022", 'label_columns'] = str(['Odor'])

regression_datasets = file_df[file_df['task'] == 'regression']['dataset'].to_list()
binary_datasets = file_df[file_df['task'] == 'binary']['dataset'].to_list()

In [ ]:
file_df

In [ ]:
# Include OPERA vapor pressure dataset? Regression task
# IsomericSMILES in gslf by merging leffingwell? 
# Look more into other mixture datasets


In [ ]:

new_descriptor_labels = {}
#for _, dataset in file_df[file_df['dataset'] == 'ifra_2019'].iterrows():
for _, dataset in file_df.iterrows():
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()

    print(f"Dataset cleaning report for {dataset['dataset']}")
    df = pd.read_csv(dataset['unclean'])

    if 'nonStereoSMILES' in df.columns:
        df.rename(columns={'nonStereoSMILES': "IsomericSMILES"}, inplace=True)

    descriptor_list = literal_eval(dataset['label_columns'])

    # Remove undesirable SMILES first
    df_filtered = remove_nan_smiles(df)
    df_filtered = remove_undesirable_elements(df_filtered)
    df_filtered = remove_duplicate_smiles(df_filtered)
    df_filtered = remove_salts_and_charges_and_multimolecule(df=df_filtered)
    df_filtered = remove_molecular_weights(df=df_filtered, column_list=descriptor_list, lower=20, higher=600)

    # Then filter descriptors
    print("--- COLUMN PROCESSING ---")
    if dataset['dataset'] not in regression_datasets:
        print(f"\t{dataset['dataset']} is a classification dataset. Cleaning columns")
        df_filtered, descriptor_list = remove_low_count_columns(df=df_filtered, column_list=descriptor_list, threshold=20)
        df_filtered, descriptor_list = lowercase_and_merge_labels(df_filtered, descriptor_list)
        if dataset['dataset'] not in binary_datasets:
            df_filtered, descriptor_list = remove_no_labels(df_filtered, descriptor_list)
    else:
        print(f"\t{dataset['dataset']} is a regression dataset. Columns are untouched")

    print(f"--- DATASET CLEANING COMPLETE ---")
    print(f"\tFinal size of dataframe: {len(df_filtered)}")

    df_filtered.to_csv(f"{dataset['dataset']}/{dataset['dataset']}_combined.csv", index=False)

    output = sys.stdout.getvalue()

    with open(f"{dataset['dataset']}/{dataset['dataset']}_cleaning_report.txt", 'w') as file:
        file.write(output)

    # Restore the original stdout
    sys.stdout = old_stdout

    # Find ways to automake the labels and descriptor numbers
    new_descriptor_labels[dataset['dataset']] = {'new_label_columns': str(descriptor_list),
                                                'task_dim': len(descriptor_list),
                                                'n_datapoints': len(df_filtered),
                                                'features': str(['IsomericSMILES']),
                                                'validate': True}

In [ ]:
new_df = pd.DataFrame(new_descriptor_labels).T
new_df.reset_index(inplace=True)
new_df.rename(columns={'index': 'dataset'}, inplace=True)
file_df = pd.merge(file_df, new_df, on='dataset', how='left')

In [ ]:
file_df

In [ ]:
file_df.to_csv("file_cleaning_features.csv", index=False)